# Optimizing the CBCT calibration using 

## Import packages

In [ ]:
import numpy as np
from skimage.filters import median
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from skimage.io import imread
import subprocess
import json
import sys
import argparse
import numpy as np
from scipy.optimize import minimize
%matplotlib inline

import sys
sys.path.append('/Users/kaestner/git/scripts/python/')
import amglib.readers as rd

## Set paths

In [ ]:
muhpath  = "/Users/kaestner/git/build-imagingsuite/Release/MuhRec.app/Contents/MacOS/MuhRec" 
datapath = "/Users/Shared/Data/P20170229_Snehota/"
recon_config = "/Users/Shared/Data/P20240126_1_DinoCalibration/02_rawdata/"

## Data loader

In [ ]:
def get_recon(mask, first, last) :
    return rd.read_images(mask, first=first,last=last)

## Define cost functions

In [ ]:
def total_variation(img) :
    grad_x, grad_y, grad_z = np.gradient(img)

# Compute the absolute gradient (magnitude of gradient vector at each point)
    return np.sqrt(grad_x**2 + grad_y**2 + grad_z**2).mean()

def std_dev(img) :
    return np.std(img)

def gradient_entropy(img, bins=100):
    gx, gy, gz = np.gradient(img)
    mag = np.sqrt(gx**2 + gy**2 + gz**2)
    hist, _ = np.histogram(mag, bins=bins, density=True)
    p = hist[hist > 0]
    return -np.sum(p * np.log(p))



In [ ]:
img = get_recon(datapath+"04_evaluation/optruns/objects_{0:04d}.tif",first=300,last=331)

In [ ]:
print(total_variation(img), std_dev(img))

## Recon runner

In [ ]:
def recon_runner(muhrec_path, config, pars):
    call_info = [muhrec_path, "-f", config]
#     call_info.append(f'projections:center={pars["projections:center"]}')
#     call_info.append(f'projections:tiltangle={pars["projections:tiltangle"]}')
    call_info.append(f'projections:sod={pars["projections:sod"]}')
    call_info.append(f'projections:sdd={pars["projections:sdd"]}')
#     call_info.append(f'projections:pPoint={pars["projections:pPointx"]} {pars["projections:pPointy"]}')
      
    print("Calling muhrec with arguments:")
    for arg in call_info:
        print(arg)
        
    try:
        result = subprocess.call(call_info)
    except subprocess.CalledProcessError as e:
        print("Error Occurred:", e)

In [ ]:
pars= {
    "projections:center": 730.781,
    "projections:tiltangle": 0.2127,
    "projections:sod": 193,
    "projections:sdd": 770,
    "projections:pPointx": 1100,
    "projections:pPointy": 750
}
recon_runner(muhpath,config=datapath+"04_evaluation/ReconConfig_Initial.xml",pars=pars)

img = get_recon(datapath+"04_evaluation/optruns/objects_{0:04d}.tif",first=300,last=331)
print(total_variation(img), std_dev(img))

In [ ]:
import numpy as np
from scipy.optimize import minimize

# Initial parameters (only numeric ones for optimization)
pars= {
#    "projections:center": 730.781,
#     "projections:tiltangle": 0.2127,
#    "projections:sod": 193,
#    "projections:sdd": 770
#     "projections:pPointx": 1100,
     "projections:pPointy": 750
}

initial_pars = pars.copy()
fixed_pars = {}

bounds = [
#     (728, 733),     # bounds for projections:center
#     (-0.2, 0.2),     # projections:tiltangle
#     (150, 300),    # projections:sod
#     (600, 900),    # projections:sdd
#     (800, 1400),      # projections:pPointx
    (700, 800),    # projections:pPointy
]

# Paths
config_path = datapath + "04_evaluation/ReconConfig_Initial.xml"
recon_path_pattern = datapath + "04_evaluation/optruns/objects_{0:04d}.tif"

# Flatten
def dict_to_array(d): return np.array(list(d.values()))
def array_to_dict(arr, template): return {k: v for k, v in zip(template.keys(), arr)}



# Cost function: Run recon, load image, evaluate cost
def cost_function(arr):
    numeric_pars = array_to_dict(arr, initial_pars)
    all_pars = {**numeric_pars, **fixed_pars}

    recon_runner(muhpath, config=config_path, pars=all_pars)
    img = get_recon(recon_path_pattern, first=300, last=331)
    history["losses"].append(total_variation(img))
    history["frames"].append(img[img.shape[0]//2])
    history["pars"].append(numeric_pars)
    return losses[-1]  # or use another metric, e.g., std_dev(img)

def CBCT_optimizer(initial_pars, method, maxiter=5) :
    # Optimization
    x0 = dict_to_array(initial_pars)

    # "Nelder-Mead"
    # 'L-BFGS-B'
    # result = minimize(cost_function, x0, method='L-BFGS-B', bounds=bounds, options={"maxiter" :5})
    history ={ "losses" : [], "frames" :[], "pars" : []}
    result = minimize(cost_function, x0, method='Nelder-Mead', bounds=bounds, options={"maxiter" :5})

    # Final optimized parameter set
    optimized_pars = {**array_to_dict(result.x, initial_pars), **fixed_pars}
    print("Optimized Parameters:", optimized_pars)
    


In [ ]:
x0 = dict_to_array(initial_pars)
print(x0)

In [ ]:
result.x

In [ ]:
optimized_pars

In [ ]:
plt.plot(history['losses'])

In [ ]:
history['pars']

In [ ]:
plt.imshow(history['frames'][0]-history['frames'][-1])

In [ ]:
fig,ax = plt.subplots(1,3,figsize=[15,5])

ax[0].imshow(history['frames'][0])
ax[1].imshow(history['frames'][-1])
ax[2].hist(history['frames'][0].ravel(),bins=200,label='initial');
ax[2].hist(history['frames'][-1].ravel(),bins=200,label='final');

In [ ]:
plt.plot(history['pars']['projections:sod'])

In [ ]:
sod = []
sdd = []

for pars in history['pars'] :
    sod.append(pars['projections:sod'])
    sdd.append(pars['projections:sdd'])

In [ ]:
plt.plot(sod)
plt.plot(sdd)